In [ ]:
from collections import OrderedDict
import os
import sys

import numpy as np
from threeML import FermiGBMBurstCatalog, download_GBM_trigger_data, TimeSeriesBuilder, update_logging_level
import yaml

parent_dir = os.path.abspath("..")
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from real_data import AnalysisBuilder
from zusammen import DataSet

update_logging_level("FATAL")

In [ ]:
grb_names = ["GRB160509374", "GRB141028455"]
data_folder = "real_data/"
data_paths = {grb: data_folder + grb for grb in grb_names}

In [ ]:
z = dict(GRB160509374=1.17, GRB141028455=2.33)
source_time = dict(GRB160509374=[9,24], GRB141028455=[6,22])

### Download GRBs from Catalog

In [ ]:
det_info = dict()
gbm_catalog = FermiGBMBurstCatalog()
for grb in grb_names:
    gbm_catalog.query_sources(grb)
    det_info[grb] = gbm_catalog.get_detector_information()[grb]
det_info

In [ ]:
dload_dict = dict()
for grb, info in det_info.items():
    dload = download_GBM_trigger_data(info["trigger"], info["detectors"], data_paths[grb], compress_tte=False)

    dload_dict[grb] = OrderedDict()
    dload_dict[grb]["z"] = z[grb]
    dload_dict[grb]["source_time"] = source_time[grb]
    for det, files in dload.items():
        dload_dict[grb][str(det)] = OrderedDict()
        for kind, path in files.items():
            dload_dict[grb][str(det)][str(kind)] = str(path.absolute())

In [ ]:
with open(data_folder + "dload.yml", "r") as f:
    dload = yaml.load(f, Loader=yaml.SafeLoader)

In [ ]:
file_name = data_folder + "dload.yml"
with open(file_name, "w") as f:
    yaml.dump(dload_dict, stream=f, default_flow_style=False)

In [ ]:
dload_dict[grb_names[0]]

In [ ]:
%matplotlib widget
for grb in grb_names:
    detector = det_info[grb]["detectors"][1]
    ts_cspec = TimeSeriesBuilder.from_gbm_cspec_or_ctime(
            detector, cspec_or_ctime_file=dload_dict[grb][detector]["cspec"], rsp_file=dload_dict[grb][detector]["rsp"]
        )

    print(grb)
    ts_cspec.view_lightcurve(-10,60)

### Analysis

In [ ]:
ab = AnalysisBuilder("dload.yml", data_folder, 5)
ab.write_yaml(data_folder + "data.yml")

ds = DataSet.from_yaml(data_folder + "data.yml")
ds.to_hdf5_file(data_folder + "data.h5")

In [ ]:
ds = DataSet.from_yaml(data_folder + "data.yml")
ds.to_hdf5_file(data_folder + "data.h5")